In [1]:
import networkx as nx
from networkx.algorithms.community.modularity_max import greedy_modularity_communities
import matplotlib.pyplot as plt
from random import choices, sample
from scipy import stats
import community, pickle

In [ ]:
# First let us test the effect of polarized sides

In [ ]:
G_small_bubbles = nx.connected_caveman_graph(2, 10)
G_medium_bubbles = nx.connected_caveman_graph(2, 50)
G_large_bubbles = nx.connected_caveman_graph(2, 100)

In [ ]:
nx.draw(G_small_bubbles, node_size = 50, alpha=0.8)

In [ ]:
nx.draw(G_medium_bubbles, node_size = 50, alpha=0.8)

In [ ]:
nx.draw(G_large_bubbles, node_size = 50, alpha=0.8)

In [ ]:
dict_edgebetweenness_small = nx.edge_betweenness_centrality(G_small_bubbles, k=None, normalized=True)
dict_edgebetweenness_medium = nx.edge_betweenness_centrality(G_medium_bubbles, k=None, normalized=True)
dict_edgebetweenness_large = nx.edge_betweenness_centrality(G_large_bubbles, k=None, normalized=True)

In [2]:
def BBC_score(graph, dict_edges):
    
    # Graph partition
    #c = list(greedy_modularity_communities(graph))
    #left_partition_users = list(c[0])
    #right_partition_users = list(c[1])
    
    #partition = community.best_partition(graph)
    #print(partition)
    #pos = nx.spring_layout(graph)
    #plt.figure(figsize=(8, 8))
    #plt.axis('off')
    #nx.draw_networkx_nodes(graph, pos, node_size=100, cmap=plt.cm.RdYlBu, node_color=list(partition.values()))
    #nx.draw_networkx_edges(graph, pos, alpha=0.3)
    #plt.show(graph)
    
    # Getting the edges in the cut
    eb_list = []
    
    for i in range(len(left_partition_users)):
        name1 = left_partition_users[i]
    
        for j in range(len(right_partition_users)):
            name2 = right_partition_users[j]
        
            if (graph.has_edge(name1, name2)):

                    if ((name1, name2) in dict_edges):
                        edge_betweenness = dict_edges[(name1, name2)]
                        eb_list.append(edge_betweenness)

                    else:
                        edge_betweenness = dict_edges[(name2, name1)]
                        eb_list.append(edge_betweenness)
                    
    
    #print("Length of cut: ", len(eb_list))
    #print("Length of cut/num edges", len(eb_list)*1.0/len(graph.edges))
    
    # Let us sample from the distributions
    
    cut_dist = choices(eb_list, k=10000)
    all_dist = choices(list(dict_edges.values()), k=10000)
    
    kl_divergence = stats.entropy(all_dist, cut_dist)
    
    BCC = 1-2.71828**(-kl_divergence)
    
    return BCC

In [ ]:
print("BBC for a graph with small bubbles: ", BBC_score(G_small_bubbles, dict_edgebetweenness_small))

In [ ]:
print("BBC for a graph with medium bubbles: ", BBC_score(G_medium_bubbles, dict_edgebetweenness_medium))

In [ ]:
print("BBC for a graph with large bubbles: ", BBC_score(G_large_bubbles, dict_edgebetweenness_large))

In [ ]:
SB, MB, LB = [], [], []

for _ in range(1000):
    
    SB.append(BBC_score(G_small_bubbles, dict_edgebetweenness_small))
    MB.append(BBC_score(G_medium_bubbles, dict_edgebetweenness_medium))
    LB.append(BBC_score(G_large_bubbles, dict_edgebetweenness_large))
    
plt.hist(SB)
plt.hist(MB)
plt.hist(LB)

In [ ]:
plt.hist(SB, label="Small bubbles")
plt.hist(MB, label="Medium bubbles")
plt.hist(LB, label="Large bubbles")
plt.legend()

In [ ]:
#The size of the polarized bubble affects the BBC score
# Let us next analyze the effect of cut size

In [ ]:
G_small_cut = nx.relaxed_caveman_graph(2, 50, 0.1)
G_medium_cut = nx.relaxed_caveman_graph(2, 50, 0.3)
G_large_cut = nx.relaxed_caveman_graph(2, 50, 0.5)

In [ ]:
nx.draw(G_small_cut, node_size = 50, alpha=0.8)

In [ ]:
nx.draw(G_medium_cut, node_size = 50, alpha=0.8)

In [ ]:
nx.draw(G_large_cut, node_size = 50, alpha=0.8)

In [ ]:
dict_edgebetweenness_small = nx.edge_betweenness_centrality(G_small_cut, k=None, normalized=True)
dict_edgebetweenness_medium = nx.edge_betweenness_centrality(G_medium_cut, k=None, normalized=True)
dict_edgebetweenness_large = nx.edge_betweenness_centrality(G_large_cut, k=None, normalized=True)

In [ ]:
print("BBC for a graph with small cut: ", BBC_score(G_small_cut, dict_edgebetweenness_small))
print("BBC for a graph with medium cut: ", BBC_score(G_medium_cut, dict_edgebetweenness_medium))
print("BBC for a graph with large cut: ", BBC_score(G_large_cut, dict_edgebetweenness_large))

In [ ]:
SC, MC, LC = [], [], []

for _ in range(1000):
    
    SC.append(BBC_score(G_small_cut, dict_edgebetweenness_small))
    MC.append(BBC_score(G_medium_cut, dict_edgebetweenness_medium))
    LC.append(BBC_score(G_large_cut, dict_edgebetweenness_large))
    


In [ ]:
plt.hist(SC, label = "Small cut")
plt.hist(MC, label = "Medium cut")
plt.hist(LC, label = "Large cut")
plt.legend()

In [ ]:
# The cut size definitely affects the score
# Lastly, what about the unequal sized bubbles, let us check!

In [ ]:
G_balanced = nx.relaxed_caveman_graph(2, 1000, 0.1)
#nx.draw(G_balanced, node_size = 50, alpha=0.8)

In [ ]:
dict_edgebetweenness_balanced = nx.edge_betweenness_centrality(G_balanced, k=None, normalized=True)

In [ ]:
BBC_score(G_balanced, dict_edgebetweenness_balanced)

In [ ]:
to_remove_10 = sample(range(1000, 2001), int(0.1*len(G_balanced.nodes)*0.5))
to_remove_25 = sample(range(1000, 2001), int(0.25*len(G_balanced.nodes)*0.5))
to_remove_50 = sample(range(1000, 2001), int(0.5*len(G_balanced.nodes)*0.5))
to_remove_75 = sample(range(1000, 2001), int(0.75*len(G_balanced.nodes)*0.5))

In [ ]:
G_little_unbalanced = G_balanced.copy()
G_moderately_unbalanced = G_balanced.copy()
G_very_unbalanced = G_balanced.copy()
G_super_unbalanced = G_balanced.copy()

In [ ]:
G_little_unbalanced.remove_nodes_from(to_remove_10)
G_moderately_unbalanced.remove_nodes_from(to_remove_25)
G_very_unbalanced.remove_nodes_from(to_remove_50)
G_super_unbalanced.remove_nodes_from(to_remove_75)

In [ ]:
dict_edgebetweenness_little_unbalanced = nx.edge_betweenness_centrality(G_little_unbalanced, k=None, normalized=True)
dict_edgebetweenness_moderately_unbalanced = nx.edge_betweenness_centrality(G_moderately_unbalanced, k=None, normalized=True)
dict_edgebetweenness_very_unbalanced = nx.edge_betweenness_centrality(G_very_unbalanced, k=None, normalized=True)
dict_edgebetweenness_super_unbalanced = nx.edge_betweenness_centrality(G_super_unbalanced, k=None, normalized=True)

In [ ]:
print("BBC for a graph with small dif: ", BBC_score(G_little_unbalanced, dict_edgebetweenness_little_unbalanced))
print("BBC for a graph with medium dif: ", BBC_score(G_moderately_unbalanced, dict_edgebetweenness_moderately_unbalanced))
print("BBC for a graph with large dif: ", BBC_score(G_very_unbalanced, dict_edgebetweenness_very_unbalanced))
print("BBC for a graph with super dif: ", BBC_score(G_super_unbalanced, dict_edgebetweenness_super_unbalanced))

In [ ]:
SD, MD, LD, SSD = [], [], [], []

for _ in range(200):
    
    SD.append(BBC_score(G_little_unbalanced, dict_edgebetweenness_little_unbalanced))
    MD.append(BBC_score(G_moderately_unbalanced, dict_edgebetweenness_moderately_unbalanced))
    LD.append(BBC_score(G_very_unbalanced, dict_edgebetweenness_very_unbalanced))
    SSD.append(BBC_score(G_super_unbalanced, dict_edgebetweenness_super_unbalanced))
    

In [ ]:
plt.hist(SD, label = "Little unbalanced")
plt.hist(MD, label = "Moderately unbalanced")
plt.hist(LD, label = "Very unbalanced")
plt.hist(SSD, label = "Super unbalanced", alpha=0.8)
plt.legend(loc="upper left")

## ANALYSIS OF OUR DATA

In [3]:
ht = "ilmastonmuutos"

In [4]:
#Load the network an communities

G = nx.read_gml(ht + "/" + ht +"_retweet_network_giant.gml")
print(nx.info(G))

#with open('ebdict.pickle', 'rb') as handle:
    #dict_edgebetweenness = pickle.load(handle)
    
dict_edgebetweenness = nx.edge_betweenness_centrality(G, k=None, normalized=True)
    
left_partition_users, right_partition_users = [], []

with open(ht + "/" + ht + "_community1.txt") as f1:
    lines = f1.readlines()

for line in lines:
    line = line.strip()
    left_partition_users.append(line)
    
with open(ht + "/" + ht + "_community2.txt") as f2:
    lines = f2.readlines()

for line in lines:
    line = line.strip()
    right_partition_users.append(line)

Name: 
Type: Graph
Number of nodes: 15699
Number of edges: 51558
Average degree:   6.5683


In [5]:
OG_NET = []

for _ in range(1000):
    
    OG_NET.append(BBC_score(G, dict_edgebetweenness))

In [7]:
import numpy as np
#plt.style.use('ggplot')

In [8]:
print("Mean: ", np.mean(OG_NET))
print("Standard deviation: ", np.std(OG_NET))

Mean:  0.7064279253909453
Standard deviation:  0.01585436565650774


In [ ]:
weights = np.ones_like(OG_NET)/float(len(OG_NET))

In [ ]:
fig, ax = plt.subplots(1,1)

ax.hist(OG_NET, weights=weights, color="coral", bins=15, label = ht)
ax.legend()

In [ ]:
fig.savefig(ht+"_bcc.png", dpi=200)

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 5
fig_size[1] = 3
plt.rcParams["figure.figsize"] = fig_size